<a href="https://colab.research.google.com/github/CarlosMendez1997Col/Advanced-AI-ML-DL-NN-LLM-Generative-Algorithms/blob/main/Web%20Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Development of Advanced AI-ML-DL-NN-LLM-Generative Algorithms

### `Deep, automated search for analyzing calls for proposals and tenders at the Stockholm Environment Institute (SEI)`

## Web and data Mining: Scraping, Crawling and Parsing

#### Code Developed by Carlos Mendez, Research Associate I






# Install/import libraries and packages

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install scrapy
!pip install selenium
!pip install tweepy
!pip install fastapi
!pip install uvicorn
!pip install transformers
!pip install sentence-transformers
!pip install webdriver-manager
!pip install psycopg2
!pip install elasticsearch
!pip install firecrawl
!pip install pydantic
!pip install playwright

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.7/331.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully

In [2]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from fastapi import FastAPI
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from firecrawl import FirecrawlApp
from pydantic import BaseModel, Field
from playwright.sync_api import sync_playwright

# Ministerio de Ciencia Tecnología e Innovación

In [3]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://minciencias.gov.co"

def scrape_minciencias_headers():
    url = f"{BASE_URL}/convocatorias/todas"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0 Safari/537.36"
    }
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    results = []
    for row in soup.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) >= 5:
            numero = cols[0].get_text(strip=True)

            # El título y el enlace de detalle
            link_tag = cols[1].find("a")
            titulo = link_tag.get_text(strip=True) if link_tag else cols[1].get_text(strip=True)
            rel_url = link_tag["href"] if link_tag and link_tag.has_attr("href") else None
            url_detalle = BASE_URL + rel_url if rel_url else None

            descripcion = cols[2].get_text(strip=True)
            recursos = cols[3].get_text(strip=True)
            apertura = cols[4].get_text(strip=True)

            # Capturar cualquier link a PDF dentro de la fila
            pdf_links = []
            for a in row.find_all("a", href=True):
                href = a["href"]
                if href.lower().endswith(".pdf"):
                    if href.startswith("/"):
                        href = BASE_URL + href
                    pdf_links.append(href)

            results.append({
                "numero": numero,
                "titulo": titulo,
                "descripcion": descripcion,
                "recursos": recursos,
                "fecha_apertura": apertura,
                "url_detalle": url_detalle,
                "pdfs": pdf_links
            })
    return results

if __name__ == "__main__":
    data = scrape_minciencias_headers()
    for item in data:
        print(item)


{'numero': '51', 'titulo': 'CONVOCATORIA PARA EL FORTALECIMIENTO DE CAPACIDADES DE CIENCIA, TECNOLOGÍA E INNOVACIÓN EN EL DEPARTAMENTO DE CÓRDOBA ALINEADO CON LOS RETOS ESTRATÉGICOS DE CTeI DEL PLAN BIENAL 2025-2026', 'descripcion': 'Fortalecer las capacidades de Ciencia, Tecnología e Innovación (CTeI) en el Departamento de Córdoba, mediante el desarrollo de proyectos de convergencia regional que impulsen la productividad y competitividad de acuerdo a las vocaciones territoriales dirigidas a atender las demandas territoriales de', 'recursos': '$17.154.450.476', 'fecha_apertura': 'Martes, Diciembre 30, 2025', 'url_detalle': 'https://minciencias.gov.co/node/9215', 'pdfs': []}
{'numero': '45', 'titulo': 'FORTALECIMIENTO Y CREACIÓN DE NUEVOS CENTROS E INSTITUTOS DE INVESTIGACIÓN, CENTROS DE DESARROLLO TECNOLÓGICO Y CENTROS DE CIENCIA', 'descripcion': 'Aumentar la capacidad de investigación de los Centros e Institutos de Investigación, Centros de Desarrollo Tecnológico y Centros de Ciencia,

# Ministerio de Ambiente y Desarrollo Sostenible

In [4]:
import requests
from bs4 import BeautifulSoup

def scrape_minambiente():
    url = "https://www.minambiente.gov.co/convocatorias"  # Ajustar si cambia
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    data = []
    # Seleccionamos todas las filas de la tabla
    for row in soup.select("table tr"):
        cols = row.find_all("td")
        if not cols:
            continue  # saltar filas sin celdas (ej. encabezados)

        # Ajustar índices según estructura real de la tabla
        titulo = cols[0].get_text(strip=True)
        fecha = cols[1].get_text(strip=True) if len(cols) > 1 else None
        enlace_tag = cols[0].find("a")
        enlace = enlace_tag["href"] if enlace_tag else None

        data.append({
            "fuente": "MinAmbiente",
            "titulo": titulo,
            "fecha": fecha,
            "url": enlace
        })
    return data

# Prueba rápida
if __name__ == "__main__":
    resultados = scrape_minambiente()
    for r in resultados:
        print(r)


{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 1', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-01-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 2', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-02-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 3', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-03-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 4', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-04-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 5', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-05-de-2021.pdf'}
{'fuente': 'MinAmbiente',

# Fundación Natura Colombia

In [5]:
import requests
from bs4 import BeautifulSoup

url = "https://natura.org.co/convocatorias/"
resp = requests.get(url)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

for block in soup.select("div.box-text.text-left"):
    titulo = block.find("h5", class_="post-title")
    resumen = block.find("p", class_="from_the_blog_excerpt")
    fecha = block.find("div", class_="post-date")
    if titulo:
        print("Título:", titulo.get_text(strip=True))
    if resumen:
        print("Resumen:", resumen.get_text(strip=True))
    if fecha:
        print("Fecha:", fecha.get_text(strip=True))
    print("-" * 60)

Título: Convocatoria Profesional control y aseguramiento de la calidad SINGEI – 4CNCC
Resumen: En el marco del proyecto “Cuarta Comunicación Nacional de Cambio Climático de Colombia y Primer...
------------------------------------------------------------
Título: Convocatoria Profesional en vulnerabilidad para el proyecto 4CNCC
Resumen: En el marco del proyecto “Cuarta Comunicación Nacional de Cambio Climático de Colombia y Primer...
------------------------------------------------------------


# Fundación para la Conservación y el Desarrollo

In [6]:
import requests
from bs4 import BeautifulSoup

url = "https://fcds.org.co/oportunidades-laborales/"
resp = requests.get(url)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

# Cada convocatoria está en un bloque con h3 y p
for block in soup.select("div.elementor-widget-container"):
    titulo = block.find("h3")
    objetivo = block.find("p")
    if titulo and objetivo:
        print("Título:", titulo.get_text(strip=True))
        print("Objetivo:", objetivo.get_text(strip=True))
        print("-" * 60)


Título: Profesional en análisis geoespacial de dinámicas de motores de transformación en bioma amazónico y fronteras
Objetivo: Prestar sus servicios profesionales para generar y analizar información geoespacial y documental relacionada a las trasformaciones territoriales, presentes en bioma amazónico nacional y sus fronteras.
------------------------------------------------------------
Título: Profesional en análisis geoespacial de dinámicas de motores de transformación en bioma amazónico y fronteras
Objetivo: Prestar sus servicios profesionales para generar y analizar información geoespacial y documental relacionada a las trasformaciones territoriales, presentes en bioma amazónico nacional y sus fronteras.
------------------------------------------------------------
Título: Consultoría Especializada en SST año 2026
Objetivo: Contratar una persona jurídica que preste servicios profesionales de outsourcing para la actualización documental, implementación y mejora continua del sistema de

# FontAgro

In [7]:
import requests
from bs4 import BeautifulSoup

url = "https://fontagro.org/es/iniciativas/convocatorias"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

convocatorias = []

# Buscar títulos
for title in soup.find_all("h4", class_="initiative-title"):
    titulo = title.get_text(strip=True)

    # Buscar el enlace asociado (si existe)
    link_tag = title.find_next("a")
    link = link_tag["href"] if link_tag else None

    # Buscar la imagen asociada
    img_tag = title.find_next("img", class_="initiative-image")
    img = img_tag["src"] if img_tag else None

    convocatorias.append({
        "titulo": titulo,
        "link": f"https://fontagro.org{link}" if link else None,
        "imagen": img
    })

# Mostrar resultados
for c in convocatorias:
    print(c)



{'titulo': 'Convocatoria 2026', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2026', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2025_f0388b44d4.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2025', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2025', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2024_160edcb44a.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2024', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2024', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2023_9d9acb3d59.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2023', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2023', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2022_d6c386cec2.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2022', 'link': 'https://fon

In [16]:
%pip install -q google-colab-selenium
from selenium.webdriver.common.by import By
import google_colab_selenium as gs

!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,214 B]
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,039 B in 1s (2,668 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
R

In [18]:
# Inicializar el driver
driver = gs.Chrome()  # headless por defecto

# Probar con una página sencilla
driver.get("https://www.python.org")
print("Título:", driver.title)

driver.quit()

<IPython.core.display.Javascript object>

Título: Welcome to Python.org


<IPython.core.display.Javascript object>

Welcome to Python.org
